# Automated ML with Regression model 

# On case of Taxi Data



<img src="flow2.png"
     alt="Flow"
     style="float: left; margin-right: 10px;" />

In [1]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

### Getting Taxi Data

In [2]:
green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))

green_taxi_df.head(10)

[Info] read from /tmp/tmpyy0eopee/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=1/part-00175-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2745-1.c000.snappy.parquet
[Info] read from /tmp/tmptz5hl3ps/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=2/part-00007-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2577-1.c000.snappy.parquet
[Info] read from /tmp/tmp63qju5hm/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=3/part-00133-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2703-1.c000.snappy.parquet
[Info] read from /tmp/tmp22tdlb9a/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=4/part-00073-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2643-1.c000.snappy.parquet
[Info] read from /tmp/tmpln06k35h/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=5/part-00177-tid-4753095944193949832

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
407341,1,2015-01-19 16:23:59,2015-01-19 16:27:54,1,0.60,None,None,-73.892647,40.747150,-73.893745,...,2,4.5,0.0,0.5,0.3,0.00,0.0,NaN,5.30,1.0
563534,2,2015-01-22 07:59:14,2015-01-22 08:03:04,1,0.65,None,None,-73.938538,40.818123,-73.948143,...,2,4.5,0.0,0.5,0.3,0.00,0.0,NaN,5.30,1.0
992530,1,2015-01-25 04:34:41,2015-01-25 04:56:55,2,7.50,None,None,-73.943207,40.811024,-73.955124,...,2,24.0,0.5,0.5,0.3,0.00,0.0,NaN,25.30,1.0
740754,1,2015-01-17 01:36:35,2015-01-17 01:42:28,1,1.10,None,None,-73.938255,40.817818,-73.950577,...,1,6.5,0.5,0.5,0.3,0.55,0.0,NaN,8.35,1.0
165841,2,2015-01-13 23:22:14,2015-01-13 23:38:58,1,3.07,None,None,-73.943428,40.696060,-73.894196,...,1,14.5,0.5,0.5,0.3,3.00,0.0,NaN,18.80,1.0
1400658,2,2015-01-08 00:51:07,2015-01-08 01:17:18,1,8.20,None,None,-73.922058,40.755257,-73.996933,...,1,26.5,0.5,0.5,0.3,5.40,0.0,NaN,33.20,1.0
807592,2,2015-01-01 06:25:31,2015-01-01 06:39:48,1,3.82,None,None,-73.958206,40.721661,-73.967331,...,1,14.5,0.0,0.5,0.3,3.62,0.0,NaN,18.92,1.0
503858,1,2015-01-18 05:16:38,2015-01-18 05:35:42,1,6.80,None,None,-73.951683,40.713951,-73.921387,...,1,22.0,0.5,0.5,0.3,4.65,0.0,NaN,27.95,1.0
538678,2,2015-01-18 22:57:51,2015-01-18 23:17:34,1,3.88,None,None,-73.897026,40.846870,-73.906532,...,2,15.5,0.5,0.5,0.3,0.00,0.0,NaN,16.80,1.0
443471,2,2015-01-30 22:38:30,2015-01-30 22:48:26,1,3.27,None,None,-73.890717,40.746967,-73.872360,...,1,12.5,0.5,0.5,0.3,2.76,0.0,NaN,16.56,1.0


## Creating different Time Various frames

In [3]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour

    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,month_num,day_of_month,day_of_week,hour_of_day
407341,1,2015-01-19 16:23:59,2015-01-19 16:27:54,1,0.60,None,None,-73.892647,40.747150,-73.893745,...,0.3,0.00,0.0,NaN,5.30,1.0,1,19,0,16
563534,2,2015-01-22 07:59:14,2015-01-22 08:03:04,1,0.65,None,None,-73.938538,40.818123,-73.948143,...,0.3,0.00,0.0,NaN,5.30,1.0,1,22,3,7
992530,1,2015-01-25 04:34:41,2015-01-25 04:56:55,2,7.50,None,None,-73.943207,40.811024,-73.955124,...,0.3,0.00,0.0,NaN,25.30,1.0,1,25,6,4
740754,1,2015-01-17 01:36:35,2015-01-17 01:42:28,1,1.10,None,None,-73.938255,40.817818,-73.950577,...,0.3,0.55,0.0,NaN,8.35,1.0,1,17,5,1
165841,2,2015-01-13 23:22:14,2015-01-13 23:38:58,1,3.07,None,None,-73.943428,40.696060,-73.894196,...,0.3,3.00,0.0,NaN,18.80,1.0,1,13,1,23
1400658,2,2015-01-08 00:51:07,2015-01-08 01:17:18,1,8.20,None,None,-73.922058,40.755257,-73.996933,...,0.3,5.40,0.0,NaN,33.20,1.0,1,8,3,0
807592,2,2015-01-01 06:25:31,2015-01-01 06:39:48,1,3.82,None,None,-73.958206,40.721661,-73.967331,...,0.3,3.62,0.0,NaN,18.92,1.0,1,1,3,6
503858,1,2015-01-18 05:16:38,2015-01-18 05:35:42,1,6.80,None,None,-73.951683,40.713951,-73.921387,...,0.3,4.65,0.0,NaN,27.95,1.0,1,18,6,5
538678,2,2015-01-18 22:57:51,2015-01-18 23:17:34,1,3.88,None,None,-73.897026,40.846870,-73.906532,...,0.3,0.00,0.0,NaN,16.80,1.0,1,18,6,22
443471,2,2015-01-30 22:38:30,2015-01-30 22:48:26,1,3.27,None,None,-73.890717,40.746967,-73.872360,...,0.3,2.76,0.0,NaN,16.56,1.0,1,30,4,22


### Remove unneeded columns and clean data

In [4]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID",
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)

green_taxi_df.head(5)

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
407341,1,1,0.60,-73.892647,40.747150,-73.893745,40.752750,5.30,1,19,0,16
563534,2,1,0.65,-73.938538,40.818123,-73.948143,40.821327,5.30,1,22,3,7
992530,1,2,7.50,-73.943207,40.811024,-73.955124,40.742214,25.30,1,25,6,4
740754,1,1,1.10,-73.938255,40.817818,-73.950577,40.806129,8.35,1,17,5,1
165841,2,1,3.07,-73.943428,40.696060,-73.894196,40.681606,18.80,1,13,1,23


### Check some descriptive statistics

In [5]:
green_taxi_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.782000,1.390292,2.868821,-73.836669,40.695456,-73.850888,40.701937,14.766242,6.500000,15.169583,3.235833,13.578125
std,0.412896,1.072775,3.043239,2.698319,1.488126,2.478995,1.367261,11.537301,3.452124,8.446097,1.961728,6.787929
min,1.000000,0.000000,0.000000,-74.193604,0.000000,-74.208000,0.000000,-351.800000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.060000,-73.959229,40.699739,-73.967249,40.700525,8.000000,3.750000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.900000,-73.945251,40.746883,-73.944504,40.747936,11.300000,6.500000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.630000,-73.917082,40.803762,-73.909351,40.792279,17.800000,9.250000,22.000000,5.000000,19.000000
max,2.000000,6.000000,168.620000,0.000000,40.955685,0.000000,42.465805,259.000000,12.000000,30.000000,6.000000,23.000000


### Finalize the data

In [6]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

### Look into data

In [7]:
final_df.describe()

,vendorID,passengerCount,tripDistance,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,23209.000000,23209.000000,23209.000000,23209.000000,23209.000000,23209.000000,23209.000000,23209.000000
mean,1.783446,1.393468,2.934003,14.780685,6.508337,15.179801,3.243655,13.592055
std,0.411905,1.077864,2.818023,10.253866,3.451145,8.446245,1.961289,6.795892
min,1.000000,1.000000,0.250000,0.010000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.110000,8.190000,4.000000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.980000,11.600000,7.000000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.700000,17.800000,10.000000,22.000000,5.000000,19.000000
max,2.000000,6.000000,30.000000,211.000000,12.000000,30.000000,6.000000,23.000000


## Configure WorkSpace

In [8]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

### Split the data into train and test sets


In [9]:
from sklearn.model_selection import train_test_split

y_df = final_df.pop("totalAmount")
x_df = final_df

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

## Automatically train a model

#### Setup the logging and specify:
Property 	Value in this tutorial 	Description
iteration_timeout_minutes 	2 	Time limit in minutes for each iteration. Reduce this value to decrease total runtime.
iterations 	20 	Number of iterations. In each iteration, a new machine learning model is trained with your data. This is the primary value that affects total run time.
primary_metric 	spearman_correlation 	Metric that you want to optimize. The best-fit model will be chosen based on this metric.
preprocess 	True 	By using True, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.)
verbosity 	logging.INFO 	Controls the level of logging.
n_cross_validations 	5 	Number of cross-validation splits to perform when validation data is not specified.

In [10]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "iterations": 20,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

#### Parametrize AutoConfingML

In [11]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             X=x_train.values,
                             y=y_train.values.flatten(),
                             **automl_settings)

### Train the automatic regression model

In [12]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "taxi-experiment")
local_run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
taxi-experiment,AutoML_38362b20-7ffe-4d1d-a924-226bee893a63,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high ca

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Explore the results

Explore the results of automatic training with a Jupyter widget. The widget allows you to see a graph and table of all individual run iterations, along with training accuracy metrics and metadata. Additionally, you can filter on different accuracy metrics than your primary metric with the dropdown selector.

In [13]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### And retrieve the best model

In [14]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: taxi-experiment,
Id: AutoML_38362b20-7ffe-4d1d-a924-226bee893a63_18,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
                                             PreFittedSoftVotingRegressor(estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmregressor', LightGBMRegressor(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('16', Pipeline(memory=None, steps=[('robustscaler', RobustScaler(copy=True, qu

### Test the best model accuracy

Use the best model to run predictions on the test data set to predict taxi fares. The function predict uses the best model and predicts the values of y, trip cost, from the x_test data set. Print the first 10 predicted cost values from y_predict.

In [15]:
y_predict = fitted_model.predict(x_test.values)
print(y_predict[:10])

[30.97032243  6.31474416  8.20309376 13.10023466 24.71177551 10.88774902
 11.85201005  8.52708824 33.96421428  9.5464549 ]


In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

5.088778719443321

### Calculating MAPE

In [17]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.14200137957215342

Model Accuracy:
0.8579986204278466


From the two prediction accuracy metrics, you see that the model is fairly good at predicting taxi fares from the data set's features, typically within +- $4.00, and approximately 15% error.

The traditional machine learning model development process is highly resource-intensive, and requires significant domain knowledge and time investment to run and compare the results of dozens of models. Using automated machine learning is a great way to rapidly test many different models for your scenario.